In [4]:
import os
import cv2
import time
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader , Dataset , sampler
import matplotlib.pyplot as plt
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.pytorch import ToTensorV2

ModuleNotFoundError: No module named 'albumentations.torch'

In [2]:
from dicom2png import dicom_to_dict

In [3]:
rle_df = pd.read_csv('/Users/hendrichscullen/Desktop/Project/EDA_Pneum_Dataset/src_data/siim/train-rle.csv')
rle_df.columns = ['ImageId', 'EncodedPixels']

In [4]:
train_fns = sorted(glob('/Users/hendrichscullen/Desktop/Project/EDA_Pneum_Dataset/src_data/siim/dicom-images-train/*/*/*.dcm'))

train_metadata_df = pd.DataFrame()
train_metadata_list = []


for file_path in tqdm(train_fns):
    dicom_data = pydicom.dcmread(file_path)
    train_metadata = dicom_to_dict(dicom_data, file_path, rle_df)
    train_metadata_list.append(train_metadata)

train_metadata_df = pd.DataFrame(train_metadata_list)
    

100%|██████████| 12089/12089 [00:59<00:00, 203.22it/s]


In [5]:
test_fns = sorted(glob('/Users/hendrichscullen/Desktop/Project/EDA_Pneum_Dataset/src_data/siim/dicom-images-test/*/*/*.dcm'))

test_metadata_df = pd.DataFrame()
test_metadata_list = []


for file_path in tqdm(test_fns):
    dicom_data = pydicom.dcmread(file_path)
    test_metadata = dicom_to_dict(dicom_data, file_path, rle_df)
    test_metadata_list.append(test_metadata)

test_metadata_df = pd.DataFrame(test_metadata_list)
    

100%|██████████| 3205/3205 [00:19<00:00, 167.57it/s]


In [6]:
import tensorflow as tf
import cv2
from skimage import morphology, io , color , exposure , img_as_float , transform


In [7]:
model_dir = '/Users/hendrichscullen/Desktop/Project/EDA_Pneum_Dataset/src_data/trained_model.hdf5'
lung_seg_model = tf.keras.models.load_model(model_dir, custom_objects= None, compile = True)

2021-12-13 16:53:48.387275: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
seg_size = 256

In [9]:
from structure_period import bounding_box, get_lung_seg_rle, remove_small_holes , get_lung_seg_tensor , remove_small_region , plot_lung_seg

In [13]:
train_lung_mask_df = get_lung_seg_rle(train_metadata_df, seg_size , lung_seg_model)
# train_lung_mask_df.to_csv('./train_lung_mask.csv', index= False)

In [11]:
test_lunng_mask_df = get_lung_seg_rle(test_metadata_df, seg_size , lung_seg_model)
# test_lunng_mask_df.to_csv('./test_lung_mask.csv', index= False)

KeyboardInterrupt: 

In [12]:
train_lm_metadata_df = pd.concat([train_metadata_df, train_lung_mask_df.drop('id', axis = 1)], axis= 1)
test_lm_metadata_df = pd.concat([test_metadata_df, test_lunng_mask_df.drop('id', axis= 1)], axis = 1)

NameError: name 'train_lung_mask_df' is not defined

In [ ]:
for i , r in train_lm_metadata_df[train_lm_metadata_df['has_pneumothorax'] == True][:10].iterrows():
    
    file_path = r['file_path']
    encoded_pixels_list = r['encoded_pixels_list']
    lung_mask = r['lung_mask']
    rmin = r['rmin']
    rmax = r['rmax']
    cmin = r['cmin']
    cmax = r['cmax']


    plot_lung_seg(file_path, encoded_pixels_list, lung_mask , rmin , rmax , cmin , cmax)